In [38]:
import codecs
import json
import pandas as pd

In [39]:

with open('mapping.manual.json',encoding='utf-8') as f:
    en_to_ar = json.loads(f.read())

NWORDS = {}
word_counts = codecs.open('ar.txt', encoding='utf8').read().split("\n")
for word_count in word_counts:
    if word_count:
        [word, n] = word_count.split()
        NWORDS[word] = int(n)


In [40]:
def sort_by_frequency(words):
    sor = {}
    for w in words:
        sor[w] = NWORDS.get(w,1)
    return sorted(words, key=sor.__getitem__, reverse=True)


def transliterate_word(english):
  ret = set()

  def recur(letters, word, start=False):
    if len(letters) == 0:
      ret.add(word)
      return
    if start:
      table = en_to_ar['start']
    else:
      table = en_to_ar['other']
    max_key_len = len(max(list(table), key=len))
    for i in range(1, max_key_len + 1):
      l = letters[:i]
      if l in table:
        for ar in table[l]:
          recur(letters[i:], word + ar)

  recur(english, '', True)
  return ret

def transliterate(sentence, verbose=False):
    sentence = clear(sentence)
    words = sentence.split()
    ret = []
    for word in words:
        try: 
            candidates = list(transliterate_word(word))
            if verbose:
                for word in candidates:
                    print (word)
            ret.append(sort_by_frequency(candidates)[0])
        except IndexError:
            ret.append(word)

    return ' '.join(ret)


In [101]:
from nltk.tokenize import WhitespaceTokenizer
import re
import itertools

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

def clear(sentence):
    sentence = emoji_pattern.sub(r'',sentence)
    sentence = re.sub(r'^RT', '', sentence)
    sentence = re.sub(r'\&\w*;', '', sentence)
    sentence = re.sub(r'\@\w*', '', sentence)
    sentence = re.sub(r'\$\w*', '', sentence)
    sentence = re.sub(r'https?:\/\/.*\/\w*', '', sentence)
    sentence = re.sub(r'#\w*', '', sentence)
    sentence = re.sub(r'[.]+', '', sentence)
    sentence = re.sub(r'[\t,\.!?`()\-$*&؟\"\\\'،%~§+]', '', sentence)        
    sentence = re.sub(r'[»]', ' ', sentence)
    sentence = sentence.lower()
    sentence = sentence.strip(' ')
    sentence = re.sub('  ', ' ', sentence)
    sentence = ''.join(ch for ch, _ in itertools.groupby(sentence))
    return sentence

print(clear("2thhab ()"))

print(clear('مناش متربين ربي روحك قبل جاي تسيب في عسولك  علينا  😂إنتي أمسط واحد في العالم 😅😅😅😅😅😅😅😅😅😅........'))



2thab
مناش متربين ربي روحك قبل جاي تسيب في عسولك علينا إنتي أمسط واحد في العالم


In [92]:
print (transliterate('qlto laka 2thhab men hona 2mok tab7atho 3ank') )

قلت لك أذهب من هنا أمك تبحث عنك


In [26]:
print(transliterate(clear('صراحة bravo أحمد الرحموني')))

صراحة برافو أحمد الرحموني


In [29]:
print(transliterate(clear('"امسط منها مافماش موش خارج عليها دور جمله _x000D_')))

"امسط منها مافماش موش خارج عليها دور جمله _x0d_


In [44]:
print (transliterate('qlto laka 2thhab men hona 2mok tab7atho 3ank') )

قلت لك أذهب من هنا أمك تبحث عنك


In [102]:
tn =pd.read_csv('tn_corp.txt',encoding='utf-8',sep="|",error_bad_lines=False)

tn.sample(5)

,sentence
6904,وحدو في الهشاشة
3988,بليدمان :-B
2887,شحال تحشوفيه والله ياخي كيما تحبوش الفساد والر...
736,T7founa 😚
1218,محلاك عسل


In [ ]:

tn['sentence']  = tn['sentence'].apply(transliterate)

tn["class"] = "tunisien"

tn.sample(5)

In [ ]:
tn.to_csv("data_5.csv", sep=';', encoding='utf-8')